In [7]:
import pandas as pd
import numpy as np

In [8]:
users = pd.read_csv(r'dataset/user.csv')

chip_type = pd.read_csv("dataset/chip_type.csv")

In [9]:
import string
import random

In [10]:
'''CREATE TABLE chip
(CHIP_NAME  VARCHAR(20),
CHIP_VERSION  VARCHAR(20),
PACKAGE_ID  INT,
CHIP_ID  INT,
FOREIGN KEY(CHIP_NAME,CHIP_VERSION) REFERENCES chip_type(CHIP_NAME,CHIP_VERSION),
FOREIGN KEY(PACKAGE_ID) REFERENCES package(PACKAGE_ID));'''
package_id = np.random.choice([i for i in range(150)]+[i for i in range(150,500)],
                                size=800, p=[0.5/150]*150+[0.5/350]*350)
chip = chip_type.sample(n=800, random_state=0,replace=True)
chip_id = [i for i in range (800)]
chip['package_id'],chip['chip_id']=package_id,chip_id
chip.reset_index(inplace=True,drop=True)
chip.to_csv('dataset/chip.csv',index=False)

In [16]:
'''CREATE TABLE package
(USER_ID   INT,
PACKAGE_ID  INT,
BUDGET  DEC(6,2),
CREATE_TIME DATETIME,
DEADLINE  DATETIME,
PRIMARY KEY(PACKAGE_ID),
FOREIGN KEY(USER_ID) REFERENCES user(USER_ID));'''
chip = pd.read_csv(r'dataset/chip.csv')
def findSmallest(arr):
    smallest = arr[0]
    smallest_index = 0
    for i in range(1,len(arr)):
        if arr[i] < smallest:
            smallest = arr[i]
            smallest_index = i
    return smallest_index
def sort(arr):
    new_arr = []
    for i in range(len(arr)):
        smallest = findSmallest(arr)
        new_arr.append(arr[smallest])
        arr = np.delete(arr,smallest)
    return new_arr
#get all the package id from the chip table
#sort the package id
package_id = chip["package_id"]
chip_price = [0]*len(chip)
for i in range(len(chip)):
    chip_n = chip.iloc[i]["chip_name"]
    chip_v = chip.iloc[i]["chip_version"]
    for j in range(len(chip_type)):
        chip_pr = chip_type.iloc[j]["price"]
        if chip_type.iloc[j]["chip_name"] == chip_n:
            if chip_type.iloc[j]["chip_version"] == chip_v:
                chip_price[i] = chip_pr
package_id_m = package_id.unique()

user = pd.read_csv("dataset/user.csv")
role_dict = dict()
for _, rows in user.iterrows():
    role_dict[int(rows["user_id"])] = rows["role"]

user_id = []
for i in range(package_id_m.size):
    user_id.append(random.randint(1, 1062))
    while role_dict[user_id[len(user_id) - 1]] == "staff":
        user_id[len(user_id) - 1] = random.randint(1, 1062)

package_id_new = sort(package_id_m)
#calculate the budget for different package
budget = [0]*len(package_id_new)
for i in range(len(package_id_new)):
    for j in range(len(package_id)):
        if(package_id_new[i]==package_id[j]):
            budget[i] += chip_price[j]


375

In [17]:
import datetime
import time
 
end_time=datetime.datetime.now()
start_time=datetime.datetime.now() + datetime.timedelta(days=-365) # 当前时间减去3分钟
 
a1=tuple(start_time.timetuple()[0:9])    #设置开始日期时间元组（2020-04-11 16:30:21）
a2=tuple(end_time.timetuple()[0:9])   #设置结束日期时间元组（2020-04-11 16:33:21）
 
start=time.mktime(a1)    #生成开始时间戳
end=time.mktime(a2)      #生成结束时间戳

create_time = []
deadline = []
#随机生成日期字符串
for i in range(len(user_id)):
    t=random.randint(start,end)    #在开始和结束时间戳中随机取出一个
    date_touple=time.localtime(t)          #将时间戳生成时间元组
    date=time.strftime("%Y-%m-%d %H:%M:%S",date_touple)   #将时间元组转成格式化字符串（2020-04-11 16:33:21）
    date = datetime.datetime.strptime(date,"%Y-%m-%d %H:%M:%S")
    create_time.append(date)

    duration = random.randint(10,20)
    ddl = date + datetime.timedelta(days=duration)
    # ddl = datetime.datetime.strptime(date,"%Y-%m-%d %H:%M:%S") + datetime.timedelta(days=duration)
    # ddl = time.strftime("%Y-%m-%d %H:%M:%S",ddl)
    deadline.append(ddl)

In [18]:
package = pd.DataFrame(data={'user_id':user_id,'package_id':package_id_new,
            'budget':budget,'create_time':create_time,'deadline':deadline})
package.to_csv('dataset/package.csv',index=False)

In [19]:
package_w_plant = pd.read_csv(r'dataset/plant_with_package.csv')
users = pd.read_csv(r'dataset/user.csv')
plants = pd.read_csv(r'dataset/plant.csv')
user_with_package = pd.read_csv(r'dataset/package.csv')
#users location
users_location = users[["user_id","country_name","province"]]
#plant location
plant_location_id = plants[["plant_id","country_name","province"]]
#user_with_package
u_package = user_with_package[["user_id","package_id"]]

num_package = len(u_package)

seed_package_to_user = pd.DataFrame()
seed_package_to_user = pd.merge(u_package,users_location,how="left")
# for i in range(len(u_package)):
#     u_id = u_package.iloc[i]["user_id"]
#     for j in range(len(users_location)):
#         if users_location.iloc[j]["user_id"] == u_id:
#             users_location = users_location.copy()
#             seed_package_to_user[i] = users_location.iloc[j]


In [20]:
package_with_location = seed_package_to_user
package_with_location

user_id  package_id country_name       province
0        836           0        China       Zhejiang
1        276           1        China   Heilongjiang
2        637           2        China       Shandong
3        253           3        China        Qinghai
4        798           4        China       Zhejiang
..       ...         ...          ...            ...
371      563         492        China        Sichuan
372      392         494      Namibia  International
373      160         495        China          Inner
374      592         497        China       Zhejiang
375       23         499       Poland  International

[376 rows x 4 columns]

In [21]:
package_plant_with_location = pd.merge(package_w_plant,plant_location_id,how='left')
package_plant_with_location

package_id  plant_id country_name   province
0              0         6        China   Shandong
1              1         2        China    Beijing
2              1         9        China    Shaanxi
3              1        20        China    Sichuan
4              2         1        China    Shaanxi
...          ...       ...          ...        ...
1114         497         8        China   Liaoning
1115         497         7        China  Chongqing
1116         498        12        China    Sichuan
1117         498         4        China    Jiangsu
1118         498         8        China   Liaoning

[1119 rows x 4 columns]

In [22]:
same_province= 0
same_country = 50
different_country = 100

In [23]:
fee = [0]*num_package
for i in range(num_package):
    fees = 0
    for j in range(len(package_with_location)):
        if package_with_location.iloc[j]["package_id"] == package.iloc[i]["package_id"]:
            user_country = package_with_location.iloc[j]["country_name"]
            user_province = package_with_location.iloc[j]["province"]
            for k in range(len(package_plant_with_location)):
                if package_plant_with_location.iloc[k]["package_id"] == package.iloc[i]["package_id"]:
                    plant_country = package_plant_with_location.iloc[k]["country_name"]
                    plant_province = package_plant_with_location.iloc[k]["province"]
                    if user_country==plant_country:
                        if user_province == plant_province:
                            fees += same_province
                        else:
                            fees += same_country
                    else:
                        fees += different_country
    fee[i] = fees

In [24]:
#更新budget
pb = [0]*len(package)
package_budget = package[["budget"]]
for i in range(len(package_budget)):
    pb[i] = package_budget.iloc[i]["budget"]
for i in range(len(fee)):
    pb[i] += fee[i]

In [25]:
package = pd.DataFrame(data={'user_id':user_id,'package_id':package_id_new,
            'budget':pb,'create_time':create_time,'deadline':deadline})
package.to_csv('dataset/package.csv',index=False)

In [26]:
'''CREATE TABLE state
(PACKAGE_ID  INT,
START_TIME DATETIME,
END_TIME DATETIME,
NAME VARCHAR(20),
FOREIGN KEY(PACKAGE_ID) REFERENCES package(PACKAGE_ID));'''

state_dict = ["Waiting", "Processing", "Finished", "Cancelled"]
start_time1, end_time1, name = [], [], []
for i in range(len(package)):
    ct =  create_time[i]
    lag = random.randint(1,8)
    st = ct + datetime.timedelta(hours=lag)
    start_time1.append(st)

    ddl = deadline[i]
    lag2 = random.randint(-100,0)
    et = ddl + datetime.timedelta(hours=lag2)
    end_time1.append(et)

    randon_index = random.randint(0,3)
    st_name = state_dict[randon_index]
    name.append(st_name)
state = pd.DataFrame(data={'package_id':package['package_id'],'start_time':start_time1,'end_time':end_time1,'state_name':name})
state.to_csv('dataset/state.csv',index=False)